In [1]:
from selenium import webdriver
from spacy.matcher import Matcher
import re
import spacy
import time
import pandas as pd 

In [2]:
def onlynumbers(gimmestring):
    num = ["0","1","2","3","4","5","6","7","8","9"]
    val = ""
    for letter in gimmestring:
        if(letter in num):
            val = val+letter
    return val

In [3]:
### Funções responsáveis por efetuar uma extração mais "inteligente" das informações brutas capturadas pelo robô 
### das páginas dos diários oficiais.

## Instala e carrega o idioma português no spacy
#!python -m spacy download pt_core_news_sm ## Descomentar na primeira vez que rodar (caso não tenha o pacote já instalado)
## Depois que instalar, resetar o kernel e comentar a linha de cima

nlp = spacy.load("pt_core_news_sm")


##### FUNÇÕES PARA EXTRAÇÃO DAS INFORMAÇÕES DO CAMPO "TITLE" #####


## Extração dos campos nomeDiário, data e numeracaoPagina do campo "Title"
def extract_data_from_title(strTitle):
    
    ## Transforma o tipo str no tipo doc
    docTitle = nlp(u''+strTitle)
    
    ## Campo nomeDiario
    diaryName = strTitle.split('-')[-1]
    
    ## Campo data
    date = extract_date(docTitle)
    
    ## Campo numeracaoPagina
    page = extract_page_number(docTitle)
    
    return diaryName, date, page
    

## Extração da data no formato string
def extract_date(nlp_doc):
    matcher = Matcher(nlp.vocab)

    ## Padrão: XX/XX/XXXX
    datePattern = [{"SHAPE": "dd/dd/dddd"},]

    ## Procura pelo padrão na entrada nlp_doc
    matcher.add('DATE', None, datePattern)
    matches = matcher(nlp_doc)

    ## Verifica se foi encontrado o padrão de data e retorna a data
    for match_id, start, end in matches:
        span = nlp_doc[start:end]
        return span.text
    else:
        ## No caso de não encontrar data, retorna string vazia
        return ''

## Extração do numero da página no formato string
def extract_page_number(nlp_doc):
    matcher = Matcher(nlp.vocab)

    ## Padrões: Pág. X, Pág. XX, Pág. XXX, Pág. XXXX
    pageNumberPattern1 = [{"LOWER": "pág"}, {"IS_PUNCT": True}, {"SHAPE": "d"}]
    pageNumberPattern2 = [{"LOWER": "pág"}, {"IS_PUNCT": True}, {"SHAPE": "dd"}]
    pageNumberPattern3 = [{"LOWER": "pág"}, {"IS_PUNCT": True}, {"SHAPE": "ddd"}]
    pageNumberPattern4 = [{"LOWER": "pág"}, {"IS_PUNCT": True}, {"SHAPE": "dddd"}]

    ## Procura pelos padrões na entrada nlp_doc
    matcher.add('PAGE_NUMBER', None, pageNumberPattern1, pageNumberPattern2, 
                pageNumberPattern3, pageNumberPattern4)
    matches = matcher(nlp_doc)

    ## Verifica se foi encontrado o padrão de página e extrai apenas o número da página
    for match_id, start, end in matches:
        span = nlp_doc[start:end]
        foundPattern = span.text
        return foundPattern.split(' ')[1]
    else:
        ## No caso de não encontrar o padrão, retorna vazio (sem página)
        return ''


    
##### FUNÇÕES PARA EXTRAÇÃO DAS INFORMAÇÕES DO CAMPO "BODY" #####

    
## Extração do decreto no formato string
def extract_decree(nlp_doc):
    matcher = Matcher(nlp.vocab)
    decreePatern1 = [{"LOWER": "art"}]

    matcher.add('DECREE_NUMBER', None, decreePatern1)

    matches = matcher(nlp_doc)
    occurrenceL = []

    for match_id, start, end in matches:
        span = nlp_doc[start:end]
        occurrenceL.append(span.text)
    return occurrenceL


## Verifica se há citação de algum decreto em cada linha de um documento.
## Se houver, então a linha é totalmente capturada.
## Entrada "conteudo" representa o campo "body"

def capture_decree_activities(content):

    activities = []

    for line in str(content).split('\nArt.'):
        
        ## Criação do documento
        document = nlp(u''+line)

        ## Verifica se algum decreto é citado na linha
        result = extract_decree(document)

        ## Caso houver citação de um decreto na linha
        if 'estabelecimento' in line:
            activities.append(line)

    return activities

In [4]:
def searchcount():
    count = driver.find_element_by_xpath("//*[@id='app-root']/div/div/div[1]/div[2]/div[1]/div[1]/div[1]")
    
    return count.text

In [5]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

path = "chromedriver.exe"

driver = webdriver.Chrome(executable_path=path, options=options)
time.sleep(3)

In [6]:
# Esta função cria uma lista com as informações dentro da página de busca do datajus

# datajus/diarios/

# title = titulo do artigo
# publisher = publicante do artigo
# body = conteúdo da página encontrada com a keyword referenciada
# link = link do artigo 

# diary = nome do diário extraído do título
# date = data de publicação do diário extraída do título
# page = página do diário extraído do título

def page_list():

    ## Lista com as informações brutas da página de busca
    dataL = []    
    
    #document_folder é a DIV dentro do HTML que representa todos os documentos dentro  da página
    #document_children serão os elementos filhos de 1º grau do elemento document_folder (cada um dos documentos)
    
    #usaremos isso para criar a condicional de visualização do crawler
    document_folder = driver.find_element_by_xpath("//*[@id='app-root']/div/div/div[1]/div[2]/div[2]")
    document_children = document_folder.find_elements_by_xpath("//*[@id='app-root']/div/div/div[1]/div[2]/div[2]/*")
    
    for i in document_children:
        print()
    
    x = 1
    for i in document_children:
        #se o atributo "class" do elemento filho se chamar 'searchpaginator', signfica que ele é o page_selector
        if((i.get_attribute('class')) == 'Pagination pagination SearchPaginator'):
            break
        publisher = driver.find_element_by_xpath("//*[@id='app-root']/div/div/div[1]/div[2]/div[2]/div[{}]/div/div[1]".format(x))
        
        #Este filtro existe, pois, durante o processo de pesquisa, entendemos que os diários dos Tribunais e Justiça
        #não tem conteúdo técnico jurídico relevante para o tema dos estabelecimentos 
        if 'Tribunal' not in publisher.text and 'Justiça' not in publisher.text:
            title = driver.find_element_by_xpath("//*[@id='app-root']/div/div/div[1]/div[2]/div[2]/div[{}]/div/h2".format(x))
            body = driver.find_element_by_xpath("//*[@id='app-root']/div/div/div[1]/div[2]/div[2]/div[{}]/div/div[2]".format(x))
            link = driver.find_element_by_xpath("//*[@id='app-root']/div/div/div[1]/div[2]/div[2]/div[{}]/div/h2/a".format(x))
            link_text = link.get_attribute("href")

            ## Extração dos campos NomeDiario, Data e Página
            diary, date, page = extract_data_from_title(title.text)
            
            dataL.append([diary, date, page, link_text, body.text])
        x = x+1
    
    return dataL

In [7]:
# esta celula ainda está sujeita a alteração, já que a KEYWORD pode ser alterado
searchUrl = 'https://www.jusbrasil.com.br/diarios/'

# dentro de KEYWORD, se coloca a palavra a ser pesquisada
# para usar mais de um termo em KEYWORD, usar mesmas regras do google:
# "covid +suspensão" (encontra artigos que contenham o texto "covid" e também o texto "suspensão")
keyword = "covid+estabelecimento+decreta"

# filt representa a ordem que os resultados serão mostrados, note que o número de dias aqui é parametrizavel 
filt = "data&l=1dia" # o número de dias aqui é parametrizável

In [8]:
# loop para entrar em cada uma das páginas do datajus

# Este loop ainda não está completo, por isso deixamos o range baixo (apenas para retirar o dataframe da primeira pagina)
data = []

df_all = pd.DataFrame()

#range maximo de 50 paginas
for i in range(1,50): ## ALTERAR ESSE RANGE DEPOIS
    page = i
    search = "busca?q="+keyword+"&p="+str(page)+"&o="+filt
    if(page==1):
        search = "busca?q="+keyword+"&o="+filt
    
    driver.get(searchUrl+search)
    
    if(page==1):
        #searchcount é o numero de resultados retornados pelo datajus
        result = searchcount()
        
    if(len(df_all)==result):
        df_all.reset_index(drop=True)
        break

    time.sleep(2)
    
    infoL = page_list()
    
    #loop dentro de cada um dos links que estão dentro da página, para entrar no artigo e extrair os dados
    count=0
    for i in range(len(infoL)):
        diary, date, pageNumber, url, body = infoL[i]
        
        time.sleep(2)
        driver.get(url)
        body = driver.find_element_by_xpath("//*[@id='app-root']/div/div/div[1]/div/div[2]/div/div[1]")
        
        ## Extração dos campo Atividade
        activities = capture_decree_activities(body.text)
        
        ## Gera lista de dados para cada par (decreto, atividade), pois um mesmo
        ## documento pode citar mais de um decreto
        for i in range(len(activities)):
            data.append([diary, date, pageNumber, url, activities[i]])        
   #     ----------
   #     for i in range(len(decrees)):
   #         data.append([diary, date, page, url, decrees[i], activities[i]])
   #     
        count=count+1
    
    df_all = pd.DataFrame(data, columns = ['NomeDiario', 'Data', 
                                    'Página', 'Url', 'Atividade'])

In [9]:
df_all

,NomeDiario,Data,Página,Url,Atividade
0,Associação dos Municípios do Mato Grosso do Sul,24/03/2020,17,https://www.jusbrasil.com.br/diarios/290880005...,"2º A partir do dia 23 de março de 2020, a par..."
1,Associação dos Municípios do Mato Grosso do Sul,24/03/2020,743,https://www.jusbrasil.com.br/diarios/290880731...,"1º De forma excepcional, com o único objetivo..."
2,Federação das Associações de Municípios da Pa...,24/03/2020,23,https://www.jusbrasil.com.br/diarios/290866987...,"2º. Em caráter excepcional, em razão da neces..."
3,Federação das Associações de Municípios da Pa...,24/03/2020,23,https://www.jusbrasil.com.br/diarios/290866987...,3º. Ficam excetuadas as atividades e os estab...
4,Federação das Associações de Municípios da Pa...,24/03/2020,23,https://www.jusbrasil.com.br/diarios/290866987...,4º. As atividades e os estabelecimentos comer...
...,...,...,...,...,...
702,Diário Oficial dos Municípios de Santa Catarina,23/03/2020,652,https://www.jusbrasil.com.br/diarios/290772343...,"2º Fica alterado o Art. 2º, do Decreto nº 9.2..."
703,Diário Oficial dos Municípios de Santa Catarina,23/03/2020,652,https://www.jusbrasil.com.br/diarios/290772343...,3º Fica inserido o parágrafo único ao Art. 2º...
704,Associação de Municípios Alagoanos,23/03/2020,12,https://www.jusbrasil.com.br/diarios/290701024...,Diários OficiaisAssociação de Municípios Alago...
705,Associação de Municípios Alagoanos,23/03/2020,12,https://www.jusbrasil.com.br/diarios/290701024...,2º-Ficam suspensas as aulas em todos os estab...


In [10]:
df_all.to_excel(r'Result.xlsx', )